In [ ]:
import numpy as np
from scipy.optimize import brentq

def R_model(t, rho, muE0, muEi):
    """
    Example model R(t):

    R(t) =  [exp(-sqrt(2*rho*t*muE0))  -  exp(-rho*t*muEi)]
           / [1 - exp(-sqrt(2*rho*t*(muE0+muEi)))]
           * [(muEi + muE0) / (muEi - muE0)]

    This is just your example form. If your actual model is different,
    put the correct formula here.
    """
    # Denominator:
    denom = 1.0 - np.exp(-np.sqrt(2.0) * rho * t * (muE0 + muEi))
    if abs(denom) < 1e-30:
        return np.inf  # or handle more carefully

    # Numerator:
    num = (np.exp(-np.sqrt(2.0) * rho * t * muE0)
           - np.exp(-np.sqrt(2.0) * rho * t * muEi))

    # Factor:
    factor = (muEi + muE0) / (muEi - muE0)

    return num / denom * factor

def f(t, rho, muE0, muEi, R_const):
    """
    The function we will actually pass to 'brentq'.
    We want to solve R_model(t) = R_const  =>  R_model(t) - R_const = 0.
    """
    return R_model(t, rho, muE0, muEi) - R_const

def numerical_derivative(g, x, args=(), eps=1e-8):
    """
    Compute a numerical approximation of the derivative g'(x)
    using a central difference scheme.
    g is a function: g(x, *args).
    """
    return (g(x + eps, *args) - g(x - eps, *args)) / (2.0 * eps)

# --------------------------
# Main script
# --------------------------

# 1) Known parameters for the model:
rho   = 8.9
muE0  = 28.84
muEi  = 65

# 2) The CONSTANT value of R you want to match (e.g., from measurement).
#    Suppose you "know" R = 0.37736632 (just as an example).
R_const = 0.164838122

# 3) Define a bracket [t_lower, t_upper].
#    We must choose an interval where f(t_lower) and f(t_upper) differ in sign.
t_lower = 0.001
t_upper = 0.02

# Evaluate f at bracket ends
f_lower = f(t_lower, rho, muE0, muEi, R_const)
f_upper = f(t_upper, rho, muE0, muEi, R_const)
print("Initial bracket:")
print(f"  t_lower = {t_lower}, f_lower = {f_lower}")
print(f"  t_upper = {t_upper}, f_upper = {f_upper}")

# If they do not differ in sign, try expanding or shifting bracket.
if f_lower * f_upper > 0:
    print("\nNo sign change in [t_lower, t_upper]. Expanding bracket automatically...")
    for i in range(10):  # up to 10 expansions
        # for example, expand t_lower by a factor of 0.5 and t_upper by factor of 2
        t_lower *= 0.5
        t_upper *= 2.0
        f_lower = f(t_lower, rho, muE0, muEi, R_const)
        f_upper = f(t_upper, rho, muE0, muEi, R_const)
        print(f"  Trying t_lower={t_lower}, t_upper={t_upper}")
        print(f"  f_lower={f_lower}, f_upper={f_upper}")
        if f_lower * f_upper < 0:
            print("Found bracket with sign change.\n")
            break
    else:
        # If we never break, no sign change was found
        raise RuntimeError("Could not find a sign change. The function may never cross R_const!")

# 4) Once we have a bracket with opposite signs, we can apply Brent's method.
#    Adjust solver tolerances as needed:
xtol = 1e-16
rtol = 1e-15

solution, result = brentq(
    f, t_lower, t_upper, 
    args=(rho, muE0, muEi, R_const),
    xtol=xtol, rtol=rtol, full_output=True
)

if not result.converged:
    raise RuntimeError("Root finding did not converge within given bracket and tolerances.")

# 5) Print the solution
t_root = solution
print(f"\nBrent's method converged: {result.converged}")
print(f"Root found: t = {t_root:.8f}")
print(f"f(t_root) = {f(t_root, rho, muE0, muEi, R_const):.6e}  (should be close to 0)")

# 6) Estimate the solver-based error in t
# Brent's method ensures the absolute error is < xtol (or) relative error < rtol*|t_root|
solver_est_tolerance = max(xtol, rtol * abs(t_root))
print(f"Solver's bracket-based tolerance on t: ±{solver_est_tolerance:.2e}")

# 7) If you have an uncertainty in R_const, say dR
#    we can do error propagation:  dt ~ dR / |dR_model/dt| at the root.
dR = 0.017402127
# derivative of f(t) w.r.t t => derivative of (R_model(t)-R_const)
# which is just the derivative of R_model(t)
Rprime = numerical_derivative(R_model, t_root, args=(rho, muE0, muEi))
dt_est = dR / abs(Rprime)

print(f"Numerical derivative of R_model at t_root: R'(t_root) = {Rprime:.6e}")
print(f"Given dR = ±{dR}, approximate dt = ±{dt_est:.6e}")
